In [3]:
using Pkg
# Pkg.add("Profile")
# Pkg.instantiate()

In [4]:
#= dependencies = ["DelimitedFiles", "LinearAlgebra", "Optim", "Distributions", "Test",
                "BenchmarkTools", "Random", "CSV", "DataFrames", "Distributed", "Plots"];
Pkg.add(dependencies) =#

In [5]:
using DelimitedFiles
using LinearAlgebra
using Optim
using Distributions
using Test
using BenchmarkTools
using Random
using CSV
using DataFrames
using Plots
using Profile
using Distributed

In [6]:
pwd()

"/Users/zifanyu/Documents/GitHub/BulkLMM.jl/analysis"

In [7]:
## Include the source code of BulkLMM to be tested:
include("../src/scan.jl");
include("../src/lmm.jl");
include("../src/wls.jl");
include("../src/util.jl");
include("../src/kinship.jl");
include("../src/readData.jl");
include("../src/scan_for_tests.jl")
include("../src/scan_distributed.jl") # new function

## Also include the helper functions for writing tests:
include("../test/testHelper.jl");

In [8]:
## Read in BXD data:
pheno_file = "../data/bxdData/BXDtraits.csv"
pheno = readBXDpheno(pheno_file);
geno_file = "../data/bxdData/BXDgeno_prob.csv"
geno = readGenoProb_ExcludeComplements(geno_file);

kinship = calcKinship(geno); # calculate kinship matrix from genotype data

In [17]:
m = size(pheno, 2);
p = size(geno, 2); # number of markers

In [18]:
## Consider the 7919-th trait
pheno_y = reshape(pheno[:, 7919], :, 1);

In [19]:
n_seeds = 20;

In [20]:
seeds_list = StatsBase.sample(1:100, n_seeds; replace = false);

In [21]:
(y0, X0, lambda0) = transform1(pheno_y, geno, kinship);

In [22]:
(r0, X00) = transform2(y0, X0, lambda0; reml = false);

In [26]:
# addprocs(4);

In [27]:
workers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [28]:
# rmprocs(workers())

In [29]:
@everywhere begin
    using Random
    using LinearAlgebra
    using Optim
    using Distributed
    
    ## Include the source code of BulkLMM to be tested:
    include("../src/scan.jl");
    include("../src/lmm.jl");
    include("../src/wls.jl");
    include("../src/util.jl");
    include("../src/kinship.jl");
    include("../src/readData.jl");
    include("../src/scan_for_tests.jl")
    include("../src/scan_distributed.jl")

    include("../src/parallel_helpers.jl")
end

In [34]:
LODs_by_blocks = scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true, 
    option = "by blocks", nblocks = 1000);

In [35]:
LODs_ordinary = scan_perms(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true);

In [37]:
LODs_by_nperms = scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1000, original = true, 
    option = "by nperms", ncopies = 10);

In [40]:
maxSqDiff(LODs_by_blocks, LODs_ordinary)

7.888609052210118e-29

In [56]:
@benchmark scan_perms_distributed(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true, 
    option = "by blocks", nblocks = 120)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.768 s …   2.823 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.795 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.795 s ± 38.282 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.77 s         Histogram: frequency by time        2.82 s <

 Memory estimate: 140.54 MiB, allocs estimate: 14229.

In [42]:
@benchmark scan_perms(pheno_y, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.801 s …    5.008 s  ┊ GC (min … max): 54.03% … 54.87%
 Time  (median):     4.905 s               ┊ GC (median):    54.46%
 Time  (mean ± σ):   4.905 s ± 146.754 ms  ┊ GC (mean ± σ):  54.46% ±  0.59%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.8 s          Histogram: frequency by time         5.01 s <

 Memory estimate: 13.30 GiB, allocs estimate: 74637.

In [55]:
@time begin
    for i in 1:10
        pheno_yi = reshape(pheno[:, i], :, 1);
        scan_perms_distributed(pheno_yi, geno, kinship; reml = false, nperms = 1000, rndseed = 0, original = true, 
                               option = "by blocks", nblocks = 120);
    end
end

 27.960931 seconds (136.17 k allocations: 1.377 GiB, 0.48% gc time)
